In [ ]:
## このプログラムは「CIFAR-10 - Object Recognition in Images」において
## 作成したノートブックで動作します
## ローカル環境のJupyter Notebookで作成したノートブックでも動作可能です

import numpy as np
from keras.datasets import cifar10
from keras.utils import to_categorical

def prepare_data():
    """データを用意する
    
    Returns:
      X_train(ndarray): 訓練データ(50000.32.32.3)
      X_test(ndarray) : テストデータ(10000.32.32.3)
      y_train(ndarray): 訓練データのOne-Hot化した正解ラベル(50000,10)
      y_train(ndarray): テストデータのOne-Hot化した正解ラベル10000,10)
      y_test_label(ndarray): テストデータの正解ラベル(10000)
    """
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()    
    # 訓練用とテスト用の画像データを正規化
    x_train, x_test = x_train.astype('float32'), x_test.astype('float32')
    x_train, x_test = x_train/255.0, x_test/255.0    
    # 訓練データとテストデータの正解ラベルを10クラスのOne-Hot表現に変換
    y_train, y_test = to_categorical(y_train), to_categorical(y_test)
    
    return x_train, x_test, y_train, y_test

In [ ]:
# モデルを生成する
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras        import optimizers

def make_convlayer():
    # Sequentialオブジェクト
    model = Sequential()
    # 畳み込み層1
    model.add(Conv2D(
        filters=64, kernel_size=3, padding='same',
        activation='relu',input_shape=(32,32,3)))
    # 2×2のプーリング層
    model.add(MaxPooling2D(pool_size=2))
    # 畳み込み層2
    model.add(Conv2D(
        filters=128, kernel_size=3, padding='same', 
        activation='relu'))
    # 2×2のプーリング層
    model.add(MaxPooling2D(pool_size=2))
    #畳み込み層3
    model.add(Conv2D(
        filters=256, kernel_size=3, padding='same', 
        activation='relu'))
    #2×2のプーリング層
    model.add(MaxPooling2D(pool_size=2))
    # Flatten層
    model.add(Flatten())
    # ドロップアウト
    model.add(Dropout(0.4))
    # 第7層
    model.add(Dense(512, activation='relu'))
    # 出力層
    model.add(Dense(10, activation = "softmax"))

    # オプティマイザーはAdam
    model.compile(loss="categorical_crossentropy",
                  optimizer=optimizers.adam(lr=0.001),
                  metrics=["accuracy"])    
    return model

In [ ]:
import numpy as np
from keras.callbacks import Callback
from keras import backend

class CyclicLR(Callback):
    """
    Attributes:
        lr_min(float)        : 最小学習率
        lr_max(float)        : 最大学習率
        step_size(int)       : ステップサイズ
        mode(str)            : スケーリングモード
        gamma(float)         : 指数関数的に減衰させるスケーリング関数の定数
        scale_fn(function)   : ラムダ式で定義されたスケーリング関数
        clr_iterations(float):イテレーション数(CLRの実行回数として)
        trn_iterations(float):イテレーション数(バッチの反復回数として)
        scale_mode(int)      : スケーリングモード
                               # 0: 単調な山型を描くスケーリング
                               # 1: 最大lrをサイクルごとに半分にする
                               # 2: 最大lrをサイクルごとに指数関数的に減衰
    """
    def __init__(self, lr_min, lr_max, step_size, mode, gamma=0.99994):
        """
        Parameters:
            lr_min(float): 最小学習率
            lr_max(float): 最大学習率
            step_size(int): ステップサイズ
            mode(str): スケーリングモード
            gamma(int): 指数関数的に減衰させるスケーリング関数の定数
        """
        self.lr_min = lr_min       # 最小学習率
        self.lr_max = lr_max       # 最大学習率
        self.step_size = step_size # ステップサイズ
        self.mode = mode           # スケーリングモード
        self.gamma = gamma         # スケーリング関数の定数
        self.clr_iterations = 0.   # CLRの実行回数
        self.trn_iterations = 0.   # イテレーション数
        self.history = {}          # 学習率とバッチ番号を記録するdict        
        self._init_scale(gamma)

    def _init_scale(self, gamma):
        """スケーリング関数とスケーリングモードの初期化
         Parameters:
            gamma(int): スケーリング関数の定数
        """
        # 単調な山形を描く場合のスケーリング関数
        if self.mode == 0:
            self.scale_fn = lambda x: 1.
            self.scale_mode = 'cycle'
        # 最大lrをサイクルごとに半分にするスケーリング関数
        elif self.mode == 1:
            self.scale_fn = lambda x: 1/(2.**(x-1))
            self.scale_mode = 'cycle'
        # 最大lrをサイクルごとに指数関数的に減衰させるスケーリング関数
        elif self.mode == 2:
            self.scale_fn = lambda x: gamma**(x)
            self.scale_mode = 'iterations'
  
    def clr(self):
        """
        ・CLRの学習率を計算する
        ・オプティマイザーに初期の学習率をセット
        """
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        # 単調な三角形、および最大学習率をサイクルごとに半分にする場合の学習率
        if self.scale_mode == 'cycle':
            return self.lr_min + (
                    self.lr_max - self.lr_min
                    )*np.maximum(0, (1-x))*self.scale_fn(cycle)
        # 最大学習率を指数関数的に減衰させる場合の学習率
        else:
            return self.lr_min + (
                    self.lr_max - self.lr_min
                    )*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        """訓練の開始時に呼ばれる
        
        オプティマイザーに初期の学習率をセット
        """
        logs = logs or {} # 
        
        self.losses = []  # 損失を記録するリスト
        self.lr = []      # 学習率を記録するリスト
        # オプティマイザーに初期学習率を設定
        if self.clr_iterations == 0:
            backend.set_value(self.model.optimizer.lr, self.lr_min)
        else:
            backend.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        """batchが終了すると呼ばれる
        
        ・オプティマイザーに学習率をセット
        ・処理中のバッチ番号と学習率をhistoryに記録
        """
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1
        # 現在の学習率を記録する
        self.history.setdefault(
                'lr', []).append(
                        backend.get_value(
                                self.model.optimizer.lr))
        # 現在のサイクルの回数を記録
        self.history.setdefault(
                'iterations', []).append(
                        self.trn_iterations)
        # logs:{'batch':バッチ番号}のdictからキーと値を取り出す
        for k, v in logs.items():
            # history:{'batch':バッチ番号のリスト}に現在のバッチ番号を追加
            self.history.setdefault(k, []).append(v)
        # clr()を実行して学習率をオプティマイザーにセット
        backend.set_value(self.model.optimizer.lr,
                          self.clr())

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler, Callback

def train(x_train, x_test, y_train, y_test, mode='triangular'): 
    """
    Parameters:
      x_train, x_test, y_train, y_test: 訓練及びテストデータ
      mode(int): CLRのモード(0, 1, 2)
    """
    batch_size = 128           # ミニバッチのサイズ
    iteration = 50000          # イテレーションサイズ＝データ数
    stepsize = iteration/128*4 # ステップサイズ
    lr_min = 0.0001            # 最小学習率
    lr_max = 0.001             # 最大学習率

    # CyclicLRオブジェクトを生成
    clr_triangular = CyclicLR(mode=mode,          # スケーリングモード
                              lr_min=lr_min,      # 最小学習率
                              lr_max=lr_max,      # 最大学習率
                              step_size=stepsize) # ステップサイズ
    # CyclicLRオブジェクトをリストに保存
    callbacks_list = [clr_triangular]
    # モデルを生成
    model = make_convlayer()

    # データ拡張
    datagen = ImageDataGenerator(
        width_shift_range=0.1,  # 0.1の割合でランダムに水平移動
        height_shift_range=0.1, # 0.1の割合でランダムに垂直移動
        rotation_range=10,      # 10度の範囲でランダムに回転させる
        zoom_range=0.1,         # ランダムに拡大
        horizontal_flip=True)   # 左右反転

    # エポック数
    epochs = 100
    # 学習を行う
    history = model.fit_generator(
        # 拡張データを生成
        datagen.flow(x_train, y_train, batch_size=batch_size),
        # ステップ数は画像の枚数をミニバッチのサイズで割った整数値
        steps_per_epoch=x_train.shape[0] // batch_size,
        epochs=epochs, # 学習回数
        verbose=1,     # 学習の進捗状況を出力
        # テストデータ
        validation_data=(x_test, y_test),
        callbacks=callbacks_list
        )
    
    # HistoryとCyclicLRを返す
    return history, clr_triangular

In [ ]:
# データの用意
x_train, x_test, y_train, y_test = prepare_data()

In [ ]:
%%time
# 学習を実行
history_0, clr_triangular_0 = train(x_train, x_test, y_train, y_test,
                                    mode=0)

In [ ]:
%%time
# 学習を実行
history_1, clr_triangular_1 = train(x_train, x_test, y_train, y_test,
                                    mode=1)

In [ ]:
%%time
# 学習を実行
history_2, clr_triangular_2 = train(x_train, x_test, y_train, y_test,
                                    mode=2)

In [ ]:
# 訓練データの精度をグラフ化
import matplotlib.pyplot as plt
%matplotlib inline

plt.ﬁgure(ﬁgsize=(10, 5)) # プロット図のサイズ
# 訓練データの精度をプロット
plt.plot(
    history_0.history['accuracy'], label='Tryangle_0', color='black')
plt.plot(
    history_1.history['accuracy'], label='Tryangle_1', color='blue')
plt.plot(
    history_2.history['accuracy'], label='exp_range_2', color='red')
plt.legend()         # 凡例表示
plt.grid()           # グリッド表示
plt.xlabel('Epoch')  # x軸ラベル
plt.ylabel('Train_Acc')    # y軸ラベル
plt.show()

In [ ]:
# 検証データの精度をグラフ化
import matplotlib.pyplot as plt
%matplotlib inline

plt.ﬁgure(ﬁgsize=(10, 5)) # プロット図のサイズ
# 訓練データの精度をプロット
plt.plot(
    history_0.history['val_accuracy'], label='Tryangle_0', color='black')
plt.plot(
    history_1.history['val_accuracy'], label='Tryangle_1', color='blue')
plt.plot(
    history_2.history['val_accuracy'], label='exp_range_2', color='red')
plt.legend()          # 凡例表示
plt.grid()            # グリッド表示
plt.xlabel('Epoch')   # x軸ラベル
plt.ylabel('Val_Acc') # y軸ラベル
plt.show()

In [ ]:
plt.ﬁgure(ﬁgsize=(10, 15)) # プロット図のサイズ

plt.subplot(3, 1, 1) # 4×1のグリッドの2にプロット
# 学習率をプロット
plt.plot(
    clr_triangular_0.history['lr'],
    label='Learning Rate(Tryangle_0)', color='blue')
plt.legend()                # 凡例表示
plt.grid()                  # グリッド表示
plt.xlabel('Iterations')    # x軸ラベル
plt.ylabel('Learning Rate') # y軸ラベル

plt.subplot(3, 1, 2) # 4×1のグリッドの3にプロット
# 学習率をプロット
plt.plot(
    clr_triangular_1.history['lr'],
    label='Learning Rate(Tryangle_1)', color='blue')
plt.legend()                # 凡例表示
plt.grid()                  # グリッド表示
plt.xlabel('Iterations')    # x軸ラベル
plt.ylabel('Learning Rate') # y軸ラベル

plt.subplot(3, 1, 3) # 4×1のグリッドの4にプロット
# 学習率をプロット
plt.plot(
    clr_triangular_2.history['lr'],
    label='Learning Rate(exp_range_2)', color='blue')
plt.legend()                # 凡例表示
plt.grid()                  # グリッド表示
plt.xlabel('Iterations')    # x軸ラベル
plt.ylabel('Learning Rate') # y軸ラベル
plt.show()